<h1><center><b>Large/Deep NN model</b></center></h1>

## Import and Load the Dataset

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# Load the preprocessed training and test datasets
train_data = pd.read_csv('dataset_preprocessed/LinearImput_Feature_Scale_Cap_Clean_Train.csv')
test_data = pd.read_csv('dataset_preprocessed/LinearImput_Feature_Scale_Cap_Clean_Test.csv')

In [3]:
# Separate features and target from the training dataset
X_train = train_data.drop(columns=['target'])
Y_train = train_data['target']
X_test = test_data.drop(columns=['target'])
Y_test = test_data['target']

## Build and Train the Model

In [30]:
model = Sequential()

# Input layer
model.add(Input(shape=(X_train.shape[1],)))

model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(16))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(1, activation='sigmoid'))

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

/home/siddhesh/miniconda3/envs/env1/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [16]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(Y_train), y=Y_train)
class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: 0.5520552664885389, 1: 5.302588035065444}


In [31]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping_classifier = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class_weights = {0: 1, 1: 3}  # Adjust based on imbalance ratio

model.fit(X_train, Y_train, 
          epochs=50, 
          batch_size=256, 
          validation_data=(X_test, Y_test), 
          callbacks=[lr_scheduler, early_stopping_classifier],
          class_weight=class_weights
         )

Epoch 1/50
3067/3067 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - AUC: 0.9888 - Precision: 0.7446 - Recall: 0.9743 - accuracy: 0.9656 - loss: 0.1101 - val_AUC: 0.9927 - val_Precision: 0.7745 - val_Recall: 0.9851 - val_accuracy: 0.9715 - val_loss: 0.0654 - learning_rate: 0.0010
Epoch 2/50
3067/3067 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - AUC: 0.9925 - Precision: 0.7705 - Recall: 0.9876 - accuracy: 0.9709 - loss: 0.0881 - val_AUC: 0.9931 - val_Precision: 0.7819 - val_Recall: 0.9841 - val_accuracy: 0.9726 - val_loss: 0.0639 - learning_rate: 0.0010
Epoch 3/50
3067/3067 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - AUC: 0.9928 - Precision: 0.7735 - Recall: 0.9870 - accuracy: 0.9716 - loss: 0.0855 - val_AUC: 0.9931 - val_Precision: 0.7692 - val_Recall: 0.9891 - val_accuracy: 0.9710 - val_loss: 0.0688 - learning_rate: 0.0010
Epoch 4/50
3067/3067 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - AUC: 0.9929 - Precision: 0.7762 - Recall: 0.9869 - accuracy: 0.9720 - loss: 0.0851 - val_AUC: 0.9929 - val_Precision: 0.7622 - val_Recall: 

In [13]:
model.save('tf_model/model_3.h5')

## Test the Model

In [32]:
# Make predictions on the test set
Y_pred_prob = model.predict(X_test)

8179/8179 ━━━━━━━━━━━━━━━━━━━━ 7s 851us/step


In [33]:
Y_pred = (Y_pred_prob > 0.5).astype(int).flatten()

In [34]:
# Evaluate the model
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print("\nConfusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Visualizing the confusion matrix
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Precision, Recall, F1-Score
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)
roc_auc = roc_auc_score(Y_test, Y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"ROC-AUC: {roc_auc:.2f}")

Accuracy: 97.27%

Confusion Matrix:
[[230203   6831]
 [   326  24352]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    237034
           1       0.78      0.99      0.87     24678

    accuracy                           0.97    261712
   macro avg       0.89      0.98      0.93    261712
weighted avg       0.98      0.97      0.97    261712



Precision: 0.78
Recall: 0.99
F1-Score: 0.87
ROC-AUC: 0.98
